In [3]:
import pandas as pd
import numpy as np

# 기상과풍력.csv 파일 읽기
df = pd.read_csv('기상과풍력.csv')

print(f"=== 원본 데이터 정보 ===")
print(f"데이터 형태: {df.shape}")
print(f"컬럼명: {list(df.columns)}")

# 1. 일자 컬럼에서 월 추출
df['일자'] = pd.to_datetime(df['일자'])
df['월'] = df['일자'].dt.month

# 2. 계절 컬럼 추가
def get_season(month):
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    else:  # 12, 1, 2
        return '겨울'

df['계절'] = df['월'].apply(get_season)

print(f"\n=== 월/계절 정보 ===")
print(f"월별 분포:")
print(df['월'].value_counts().sort_index())
print(f"\n계절별 분포:")
print(df['계절'].value_counts())

# 3. 풍향 컬럼을 라디안으로 변환 후 sin, cos 적용
# 풍향: 0~360도 → 0~2π 라디안 → sin, cos 값
df['풍향_sin'] = np.sin(df['풍향(16방위)'] * np.pi / 180)
df['풍향_cos'] = np.cos(df['풍향(16방위)'] * np.pi / 180)

# 4. 시간 컬럼을 라디안으로 변환 후 sin, cos 적용  
# 시간: 1~24 → 0~2π 라디안 → sin, cos 값
df['시간_sin'] = np.sin((df['시간'] - 1) * 2 * np.pi / 24)
df['시간_cos'] = np.cos((df['시간'] - 1) * 2 * np.pi / 24)

# 5. 월 컬럼을 라디안으로 변환 후 sin, cos 적용
# 월: 1~12 → 0~2π 라디안 → sin, cos 값
df['월_sin'] = np.sin((df['월'] - 1) * 2 * np.pi / 12)
df['월_cos'] = np.cos((df['월'] - 1) * 2 * np.pi / 12)

print(f"\n=== 풍향/시간/월 변환 완료 ===")
print(f"풍향(16방위) 범위: {df['풍향(16방위)'].min()}~{df['풍향(16방위)'].max()}")
print(f"풍향_sin 범위: {df['풍향_sin'].min():.3f}~{df['풍향_sin'].max():.3f}")
print(f"풍향_cos 범위: {df['풍향_cos'].min():.3f}~{df['풍향_cos'].max():.3f}")
print(f"시간 범위: {df['시간'].min()}~{df['시간'].max()}")
print(f"시간_sin 범위: {df['시간_sin'].min():.3f}~{df['시간_sin'].max():.3f}")
print(f"시간_cos 범위: {df['시간_cos'].min():.3f}~{df['시간_cos'].max():.3f}")
print(f"월 범위: {df['월'].min()}~{df['월'].max()}")
print(f"월_sin 범위: {df['월_sin'].min():.3f}~{df['월_sin'].max():.3f}")
print(f"월_cos 범위: {df['월_cos'].min():.3f}~{df['월_cos'].max():.3f}")

# 5. 기상 데이터 이상치 제거
print(f"\n=== 기상 데이터 이상치 처리 ===")
before_weather = len(df)

# 이상치 기준 정의
outlier_ranges = {
    '기온(°C)': (-40, 45),
    '강수량(mm)': (0, 350),
    '풍속(m/s)': (0, 50),
    '풍향(16방위)': (0, 360),
    '습도(%)': (0, 100),
    '증기압(hPa)': (0, 50),
    '이슬점온도(°C)': (-40, 35),
    '현지기압(hPa)': (850, 1050),
    '적설(cm)': (0, 200)
}

# 각 컬럼별 이상치 제거
for col, (min_val, max_val) in outlier_ranges.items():
    if col in df.columns:
        before_col = len(df)
        df = df[(df[col] >= min_val) & (df[col] <= max_val)]
        after_col = len(df)
        removed_col = before_col - after_col
        
        if removed_col > 0:
            print(f"   {col}: {removed_col:,}건 제거 ({min_val}~{max_val} 범위 외)")

after_weather = len(df)
weather_removed = before_weather - after_weather
print(f"기상 이상치 총 제거: {weather_removed:,}건 ({weather_removed/before_weather*100:.2f}%)")

# 6. 터빈 정보 매핑 (발전구분-호기별)
print(f"\n=== 터빈 정보 매핑 ===")

# 터빈 정보 딕셔너리 (발전구분, 호기) -> (블레이드, 정격, 커트인, 커트아웃)
turbine_info = {
    ('한경풍력', 1): {'블레이드': 36, '정격': 13, '커트인': 3, '커트아웃': 20},
    ('한경풍력', 2): {'블레이드': 45, '정격': 15, '커트인': 3.5, '커트아웃': 25},
    ('성산풍력', 1): {'블레이드': 40, '정격': 15, '커트인': 3.5, '커트아웃': 25},
    ('성산풍력', 2): {'블레이드': 40, '정격': 15, '커트인': 3.5, '커트아웃': 25},
    ('상명풍력', 1): {'블레이드': 56, '정격': 12, '커트인': 3, '커트아웃': 25},
    ('어음풍력', 1): {'블레이드': 68, '정격': 12, '커트인': 3, '커트아웃': 25},
    ('화순풍력', 1): {'블레이드': 46.5, '정격': 11, '커트인': 3, '커트아웃': 25}
}

# 터빈 정보 매핑 함수
def map_turbine_info(row):
    key = (row['발전구분'], row['호기'])
    if key in turbine_info:
        return pd.Series(turbine_info[key])
    else:
        return pd.Series({'블레이드': None, '정격': None, '커트인': None, '커트아웃': None})

# 터빈 정보 컬럼 추가
turbine_cols = df.apply(map_turbine_info, axis=1)
df = pd.concat([df, turbine_cols], axis=1)

# 매핑되지 않은 데이터 확인 및 제거
before_mapping = len(df)
df = df.dropna(subset=['블레이드', '정격', '커트인', '커트아웃'])
after_mapping = len(df)
mapping_removed = before_mapping - after_mapping

print(f"터빈 정보 매핑 완료:")
print(f"   - 매핑 전: {before_mapping:,}건")
print(f"   - 매핑 후: {after_mapping:,}건")
print(f"   - 매핑 실패 제거: {mapping_removed:,}건")

# 매핑된 터빈 정보 확인
print(f"\n=== 매핑된 터빈 정보 확인 ===")
unique_turbines = df[['발전구분', '호기', '블레이드', '정격', '커트인', '커트아웃']].drop_duplicates()
for _, row in unique_turbines.iterrows():
    print(f"   {row['발전구분']} {row['호기']}호기: 블레이드={row['블레이드']}m, 정격={row['정격']}m/s, 커트인={row['커트인']}m/s, 커트아웃={row['커트아웃']}m/s")

# 7. 발전량(kWh)이 설비용량(MW)을 초과하는 경우 제거
# 7. 발전량(kWh)이 설비용량(MW)을 초과하는 경우 제거
# 설비용량 MW → kWh 변환: MW * 1000 = kW (1시간 기준)
before_power = len(df)
df = df[df['발전량(kWh)'] <= (df['설비용량(MW)'] * 1000)]
after_power = len(df)
power_removed = before_power - after_power

print(f"\n=== 발전량 이상치 제거 ===")
print(f"발전량 > 설비용량 제거: {power_removed:,}건")

# 8. 풍속이 커트인~커트아웃 범위에 있는데 발전량이 0인 경우 제거
before_wind_check = len(df)
# 커트인 < 풍속 < 커트아웃 범위에 있으면서 발전량이 0인 경우 제거
wind_condition = (df['풍속(m/s)'] > df['커트인']) & (df['풍속(m/s)'] < df['커트아웃']) & (df['발전량(kWh)'] == 0)
df = df[~wind_condition]
after_wind_check = len(df)
wind_removed = before_wind_check - after_wind_check

print(f"커트인 < 풍속 < 커트아웃 범위에서 발전량=0인 데이터 제거: {wind_removed:,}건")

print(f"\n=== 전체 이상치 제거 결과 ===")
print(f"제거 전: {before_weather:,}건")
print(f"제거 후: {after_wind_check:,}건")
total_removed = before_weather - after_wind_check
print(f"총 제거: {total_removed:,}건 ({total_removed/before_weather*100:.2f}%)")

# 9. 불필요한 컬럼 제거 (일자, 원본 풍향, 원본 시간)
columns_to_drop = ['일자', '풍향(16방위)', '시간']
df_processed = df.drop(columns=columns_to_drop)

print(f"\n=== 컬럼 정리 완료 ===")
print(f"제거된 컬럼: {columns_to_drop}")
print(f"최종 컬럼 수: {len(df_processed.columns)}개")

# 10. 최종 데이터 저장
df_processed.to_csv('전처리완료_기상과풍력.csv', index=False, encoding='utf-8-sig')

print(f"\n✅ 전처리 완료!")
print(f"   - 저장 파일: 전처리완료_기상과풍력.csv")
print(f"   - 최종 데이터: {len(df_processed):,}건")

# 11. 최종 컬럼 목록 출력
print(f"\n=== 최종 컬럼 목록 ===")
for i, col in enumerate(df_processed.columns, 1):
    print(f"   {i:2d}. {col}")

# 12. 샘플 데이터 확인
print(f"\n=== 처리된 데이터 샘플 ===")
sample_cols = ['발전구분', '호기', '월', '계절', '발전량(kWh)', '설비용량(MW)', 
               '기온(°C)', '풍속(m/s)', '풍향_sin', '시간_sin', '블레이드', '정격', '커트인', '커트아웃']
print(df_processed[sample_cols].head(10))

# 13. 기본 통계 정보
print(f"\n=== 주요 컬럼 통계 ===")
numeric_cols = ['발전량(kWh)', '설비용량(MW)', '기온(°C)', '풍속(m/s)', 
                '풍향_sin', '시간_sin', '블레이드', '정격', '커트인', '커트아웃']
print(df_processed[numeric_cols].describe())

# 14. 데이터 품질 확인
print(f"\n=== 데이터 품질 확인 ===")
for col, (min_val, max_val) in outlier_ranges.items():
    if col in df_processed.columns:
        actual_min = df_processed[col].min()
        actual_max = df_processed[col].max()
        print(f"{col}: {actual_min:.2f} ~ {actual_max:.2f} (기준: {min_val} ~ {max_val})")

# 15. 발전구분별 최종 데이터 분포 확인
print(f"\n=== 발전구분별 최종 데이터 분포 ===")
plant_distribution = df_processed.groupby(['발전구분', '호기']).size()
for (plant, unit), count in plant_distribution.items():
    percentage = count / len(df_processed) * 100
    print(f"   {plant} {unit}호기: {count:,}건 ({percentage:.1f}%)")

# 16. 풍속 vs 발전량 관계 검증
print(f"\n=== 풍속 vs 발전량 관계 검증 ===")
for (plant, unit), group in df_processed.groupby(['발전구분', '호기']):
    cutin = group['커트인'].iloc[0]
    cutout = group['커트아웃'].iloc[0]
    
    # 커트인 이하에서 발전량 > 0인 경우
    below_cutin_gen = group[(group['풍속(m/s)'] <= cutin) & (group['발전량(kWh)'] > 0)]
    # 커트아웃 이상에서 발전량 > 0인 경우  
    above_cutout_gen = group[(group['풍속(m/s)'] >= cutout) & (group['발전량(kWh)'] > 0)]
    # 커트인~커트아웃 사이에서 발전량 = 0인 경우
    between_zero_gen = group[(group['풍속(m/s)'] > cutin) & (group['풍속(m/s)'] < cutout) & (group['발전량(kWh)'] == 0)]
    
    print(f"   {plant} {unit}호기 (커트인:{cutin}, 커트아웃:{cutout}):")
    print(f"     - 커트인 이하 발전: {len(below_cutin_gen)}건")
    print(f"     - 커트아웃 이상 발전: {len(above_cutout_gen)}건") 
    print(f"     - 발전구간 무발전: {len(between_zero_gen)}건 (제거됨)")

=== 원본 데이터 정보 ===
데이터 형태: (496560, 20)
컬럼명: ['발전구분', '호기', '일자', '시간', '발전량(kWh)', '설비용량(MW)', '연식(년)', '지점명', '날짜', '지점', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '적설(cm)']

=== 월/계절 정보 ===
월별 분포:
월
1     46152
2     41352
3     42408
4     41040
5     42408
6     39576
7     40920
8     40920
9     39600
10    40920
11    39600
12    41664
Name: count, dtype: int64

계절별 분포:
계절
겨울    129168
봄     125856
여름    121416
가을    120120
Name: count, dtype: int64

=== 풍향/시간/월 변환 완료 ===
풍향(16방위) 범위: 0.0~360.0
풍향_sin 범위: -1.000~1.000
풍향_cos 범위: -1.000~1.000
시간 범위: 1~24
시간_sin 범위: -1.000~1.000
시간_cos 범위: -1.000~1.000
월 범위: 1~12
월_sin 범위: -1.000~1.000
월_cos 범위: -1.000~1.000

=== 기상 데이터 이상치 처리 ===
   기온(°C): 3,640건 제거 (-40~45 범위 외)

=== 풍향/시간/월 변환 완료 ===
풍향(16방위) 범위: 0.0~360.0
풍향_sin 범위: -1.000~1.000
풍향_cos 범위: -1.000~1.000
시간 범위: 1~24
시간_sin 범위: -1.000~1.000
시간_cos 범위: -1.000~1.000
월 범위: 1~12
월_sin 범위: -1.000~1.000
월_cos 범위: -1.000~1.000

===